In [1]:
!git clone https://github.com/Yaswanth-B/AccessibleLLM.git

Cloning into 'AccessibleLLM'...
remote: Enumerating objects: 343, done.
remote: Counting objects: 100% (221/221), done.
remote: Compressing objects: 100% (176/176), done.
remote: Total 343 (delta 85), reused 135 (delta 36), pack-reused 122
Receiving objects: 100% (343/343), 57.57 MiB | 20.47 MiB/s, done.
Resolving deltas: 100% (93/93), done.


In [2]:
import zipfile
import os

# Path to the uploaded zip file
zip_path = '/content/AccessibleLLM/object detection/dataset3.zip'

# Directory to extract to
extract_path = '/content/AccessibleLLM/object detection/dataset3'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

os.listdir(extract_path)

['dataset3']

In [4]:
!pip install bert-score torch transformers

In [5]:
!pip install rouge

In [6]:
!pip install -U torch torchvision
!pip install -U git+https://github.com/openai/CLIP.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 MB 5.3 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 2.2.0
    Uninstalling triton-2.2.0:
      Successfully uninstalled triton-2.2.0
  Attempting uninstall: nvidia-nccl-cu12
    Found existing installation: nvidia-nccl-cu12 2.19.3
    Uninstalling nvidia-nccl-cu12-2.19.3:
      Successfully uninstalled nvidia-nccl-cu12-2.19.3
  Attempting uninstall: torch
    Found existing installation: torch 2.2.1+cu121
    Uninstalling torch-2.2.1+cu121:
      Successfully uninstalled torch-2.2.1+cu121
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.17.1+cu121
    Uninstalling torchvision-0.17.1+cu121:
      Successfully uninst

In [7]:
!pip install nltk
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [10]:
from rouge import Rouge

# Load reference captions and generated captions
with open("/content/AccessibleLLM/object detection/dataset3/dataset3/ds3text.txt", "r") as ref_file:
    reference_captions = ref_file.readlines()

with open("/content/AccessibleLLM/object detection/mode1d3.txt", "r") as model1_file:
    model1_captions = model1_file.readlines()

with open("/content/AccessibleLLM/object detection/model2d3.txt", "r") as model2_file:
    model2_captions = model2_file.readlines()

# Initialize ROUGE
rouge = Rouge()

# Compute ROUGE scores for model 1
scores_model1 = rouge.get_scores(model1_captions, reference_captions, avg=True)
print("ROUGE Scores for BLIP-2:")
print(scores_model1)

# Compute ROUGE scores for model 2
scores_model2 = rouge.get_scores(model2_captions, reference_captions, avg=True)
print("ROUGE Scores for VitGpt:")
print(scores_model2)

# Calculate precision, recall, and F1-score based on ROUGE scores
precision_model1 = scores_model1["rouge-l"]["p"]
recall_model1 = scores_model1["rouge-l"]["r"]
f1_model1 = scores_model1["rouge-l"]["f"]

precision_model2 = scores_model2["rouge-l"]["p"]
recall_model2 = scores_model2["rouge-l"]["r"]
f1_model2 = scores_model2["rouge-l"]["f"]

print("Precision, Recall, and F1-score for BLIP-2:")
print("Precision:", precision_model1)
print("Recall:", recall_model1)
print("F1-score:", f1_model1)

print("Precision, Recall, and F1-score for VitGpt:")
print("Precision:", precision_model2)
print("Recall:", recall_model2)
print("F1-score:", f1_model2)

ROUGE Scores for BLIP-2:
{'rouge-1': {'r': 0.7057404605198723, 'p': 0.5711694838529204, 'f': 0.6259112389882882}, 'rouge-2': {'r': 0.46176681935195857, 'p': 0.35838442697653206, 'f': 0.39949114886886683}, 'rouge-l': {'r': 0.6315150650003591, 'p': 0.5119797835463471, 'f': 0.560635915103244}}
ROUGE Scores for VitGpt:
{'rouge-1': {'r': 0.44236701370524906, 'p': 0.4042870038458274, 'f': 0.41503899617383466}, 'rouge-2': {'r': 0.20881120625160865, 'p': 0.16989874025183618, 'f': 0.18336638637340974}, 'rouge-l': {'r': 0.39817976304741015, 'p': 0.3620736453089395, 'f': 0.37246338708799215}}
Precision, Recall, and F1-score for BLIP-2:
Precision: 0.5119797835463471
Recall: 0.6315150650003591
F1-score: 0.560635915103244
Precision, Recall, and F1-score for VitGpt:
Precision: 0.3620736453089395
Recall: 0.39817976304741015
F1-score: 0.37246338708799215


In [13]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import bert_score

with open("/content/AccessibleLLM/object detection/mode1d3.txt", "r") as file:
    model1_text = file.read()

blip2_text = model1_text.strip().split('\n')

with open("/content/AccessibleLLM/object detection/model2d3.txt", "r") as file:
    model2_text = file.read()

VitGpt_text = model2_text.strip().split('\n')

with open("/content/AccessibleLLM/object detection/dataset3/dataset3/ds3text.txt", "r") as file:
    references_text = file.read()

true_text = references_text.strip().split('\n')

# Compute BERTScore for BLIP-2 Model
P_A, R_A, F1_A = bert_score.score(
    blip2_text,
    true_text,
    lang='en',  # Language of the captions (e.g., 'en' for English)
    model_type='bert-base-uncased',  # BERT model type
)

# Compute BERTScore for VitGpt Model
P_B, R_B, F1_B = bert_score.score(
    VitGpt_text,
    true_text,
    lang='en',  # Language of the captions (e.g., 'en' for English)
    model_type='bert-base-uncased',  # BERT model type
)

print("BERTScore for BLIP-2 Model:")
print(f"Precision: {P_A.mean():.4f}")
print(f"Recall: {R_A.mean():.4f}")
print(f"F1-score: {F1_A.mean():.4f}")

print("\nBERTScore for VitGpt Model:")
print(f"Precision: {P_B.mean():.4f}")
print(f"Recall: {R_B.mean():.4f}")
print(f"F1-score: {F1_B.mean():.4f}")


BERTScore for BLIP-2 Model:
Precision: 0.7260
Recall: 0.7872
F1-score: 0.7541

BERTScore for VitGpt Model:
Precision: 0.6246
Recall: 0.6585
F1-score: 0.6362


In [14]:
import nltk

def compute_gleu_score(generated_caption, reference_caption):
    gen_tokens = nltk.word_tokenize(generated_caption.lower())
    ref_tokens = nltk.word_tokenize(reference_caption.lower())
    gleu_score = nltk.translate.gleu_score.sentence_gleu([ref_tokens], gen_tokens)
    return gleu_score

def compute_average_gleu_scores(model_captions, reference_captions):
    total_gleu_score = 0.0
    num_captions = len(model_captions)

    for i in range(num_captions):
        model_caption = model_captions[i]
        ref_caption = reference_captions[i]
        gleu_score = compute_gleu_score(model_caption, ref_caption)
        total_gleu_score += gleu_score

    average_gleu_score = total_gleu_score / num_captions
    return average_gleu_score

def read_captions_from_file(file_name):
    with open(file_name, 'r') as file:
        captions = file.read().split('\n')
    return [caption.strip() for caption in captions if caption.strip()]

# Read reference captions from "truetext.txt"
reference_captions = read_captions_from_file("/content/AccessibleLLM/object detection/dataset3/dataset3/ds3text.txt")

# Read model captions from "model1.txt" (for Model A) and "model2.txt" (for Model B)
captions_model_a = read_captions_from_file("/content/AccessibleLLM/object detection/mode1d3.txt")
captions_model_b = read_captions_from_file("/content/AccessibleLLM/object detection/model2d3.txt")

# Compute average GLEU scores for Model A and Model B
avg_gleu_model_a = compute_average_gleu_scores(captions_model_a, reference_captions)
avg_gleu_model_b = compute_average_gleu_scores(captions_model_b, reference_captions)

# Print the results
print("Average GLEU Score for BLIP-2:", avg_gleu_model_a)
print("Average GLEU Score for VItGpt:", avg_gleu_model_b)

# Determine which model performs better
if avg_gleu_model_a > avg_gleu_model_b:
    print("BLIP-2 performs better.")
elif avg_gleu_model_b > avg_gleu_model_a:
    print("VitGpt performs better.")
else:
    print("Both models perform equally well.")


Average GLEU Score for BLIP-2: 0.3281658319708012
Average GLEU Score for VItGpt: 0.15503852724900705
BLIP-2 performs better.


In [16]:
import nltk
from nltk.translate.bleu_score import corpus_bleu

def read_captions_from_file(file_name):
    with open(file_name, 'r') as file:
        captions = file.read().split('\n')
    return [caption.strip() for caption in captions if caption.strip()]

# Read reference captions from "truetext.txt"
reference_captions = read_captions_from_file("/content/AccessibleLLM/object detection/dataset3/dataset3/ds3text.txt")

# Read model captions from "model1.txt" (for Model A) and "model2.txt" (for Model B)
captions_model_a = read_captions_from_file("/content/AccessibleLLM/object detection/mode1d3.txt")
captions_model_b = read_captions_from_file("/content/AccessibleLLM/object detection/model2d3.txt")

# Tokenize captions
model1_captions = [nltk.word_tokenize(caption.lower()) for caption in captions_model_a]
model2_captions = [nltk.word_tokenize(caption.lower()) for caption in captions_model_b]
reference_captions = [[nltk.word_tokenize(caption.lower())] for caption in reference_captions]

# Calculate BLEU score for each model
bleu_score_model1 = corpus_bleu(reference_captions, model1_captions)
bleu_score_model2 = corpus_bleu(reference_captions, model2_captions)

print("BLEU Score for BLIP-2:", bleu_score_model1)
print("BLEU Score for VitGpt:", bleu_score_model2)


BLEU Score for BLIP-2: 0.2853194240578867
BLEU Score for VitGpt: 0.09539884316244567


In [18]:
#clip score for BLIP-2
import torch
import clip
from PIL import Image
import os

def get_clip_score(image_path, text):
    # Load the pre-trained CLIP model
    model, preprocess = clip.load('ViT-B/32')

    # Preprocess the image and tokenize the text
    image = Image.open(image_path)
    image_input = preprocess(image).unsqueeze(0)
    text_input = clip.tokenize([text])

    # Move the inputs to GPU if available
    device = "cuda" if torch.cuda.is_available() else "cpu"
    image_input = image_input.to(device)
    text_input = text_input.to(device)
    model = model.to(device)

    # Generate embeddings for the image and text
    with torch.no_grad():
        image_features = model.encode_image(image_input)
        text_features = model.encode_text(text_input)

    # Normalize the features
    image_features = image_features / image_features.norm(dim=-1, keepdim=True)
    text_features = text_features / text_features.norm(dim=-1, keepdim=True)

    # Calculate the cosine similarity to get the CLIP score
    clip_score = torch.matmul(image_features, text_features.T).item()

    return clip_score

def process_images_and_texts(image_folder, text_file):
    # Get a list of image files
    image_files = [file for file in os.listdir(image_folder) if file.endswith((".jpg", ".jpeg", ".png"))]
    image_files.sort()  # Sort the list for consistent processing order

    # Read lines from the text file
    with open(text_file, 'r') as f:
        text_lines = f.readlines()

    # Process each image and text pair
    for image_file in image_files:
        # Find corresponding text line in the text file
        image_name = os.path.splitext(image_file)[0]
        for line in text_lines:
            if line.startswith(image_name):
                _, text = line.split(",", 1)  # Split by first comma to separate image name and text
                text = text.strip()  # Remove leading/trailing whitespaces
                image_path = os.path.join(image_folder, image_file)
                score = get_clip_score(image_path, text)
                print(f"CLIP Score for {image_file}: {score}")
                break
        else:
            print(f"No text found for {image_file}")

# Paths to image folder and text file
image_folder = "/content/AccessibleLLM/object detection/dataset3/dataset3"
text_file = "/content/AccessibleLLM/object detection/dataset3/dataset3/model1d3withimg.txt"

# Process images and texts
process_images_and_texts(image_folder, text_file)

CLIP Score for 1.jpg: 0.2982504665851593
CLIP Score for 10.jpg: 0.321647584438324
CLIP Score for 11.jpg: 0.30613845586776733
CLIP Score for 12.jpg: 0.2353488653898239
CLIP Score for 13.jpg: 0.3385522961616516
CLIP Score for 14.jpg: 0.32119494676589966
CLIP Score for 15.jpg: 0.24912357330322266
CLIP Score for 16.jpg: 0.3127995729446411
CLIP Score for 17.jpg: 0.28032660484313965
CLIP Score for 18.jpg: 0.30036795139312744
CLIP Score for 19.jpg: 0.23910358548164368
CLIP Score for 2.jpg: 0.2974911332130432
CLIP Score for 20.jpg: 0.3586793839931488
CLIP Score for 21.jpg: 0.3718470633029938
CLIP Score for 22.jpg: 0.31111884117126465
CLIP Score for 23.jpg: 0.304635614156723
CLIP Score for 24.jpg: 0.2570834755897522
CLIP Score for 25.jpg: 0.25392723083496094
CLIP Score for 26.jpg: 0.3001241981983185
CLIP Score for 27.jpg: 0.352206289768219
CLIP Score for 28.jpg: 0.34062421321868896
CLIP Score for 29.jpg: 0.3297742009162903
CLIP Score for 3.jpg: 0.3168848156929016
CLIP Score for 30.jpg: 0.292315

In [20]:
#clip score for vitgpt
import torch
import clip
from PIL import Image
import os

def get_clip_score(image_path, text):
    # Load the pre-trained CLIP model
    model, preprocess = clip.load('ViT-B/32')

    # Preprocess the image and tokenize the text
    image = Image.open(image_path)
    image_input = preprocess(image).unsqueeze(0)
    text_input = clip.tokenize([text])

    # Move the inputs to GPU if available
    device = "cuda" if torch.cuda.is_available() else "cpu"
    image_input = image_input.to(device)
    text_input = text_input.to(device)
    model = model.to(device)

    # Generate embeddings for the image and text
    with torch.no_grad():
        image_features = model.encode_image(image_input)
        text_features = model.encode_text(text_input)

    # Normalize the features
    image_features = image_features / image_features.norm(dim=-1, keepdim=True)
    text_features = text_features / text_features.norm(dim=-1, keepdim=True)

    # Calculate the cosine similarity to get the CLIP score
    clip_score = torch.matmul(image_features, text_features.T).item()

    return clip_score

def process_images_and_texts(image_folder, text_file):
    # Get a list of image files
    image_files = [file for file in os.listdir(image_folder) if file.endswith((".jpg", ".jpeg", ".png"))]
    image_files.sort()  # Sort the list for consistent processing order

    # Read lines from the text file
    with open(text_file, 'r') as f:
        text_lines = f.readlines()

    # Process each image and text pair
    for image_file in image_files:
        # Find corresponding text line in the text file
        image_name = os.path.splitext(image_file)[0]
        for line in text_lines:
            if line.startswith(image_name):
                _, text = line.split(",", 1)  # Split by first comma to separate image name and text
                text = text.strip()  # Remove leading/trailing whitespaces
                image_path = os.path.join(image_folder, image_file)
                score = get_clip_score(image_path, text)
                print(f"CLIP Score for {image_file}: {score}")

                break
        else:
            print(f"No text found for {image_file}")

# Paths to image folder and text file
image_folder = "/content/AccessibleLLM/object detection/dataset3/dataset3"
text_file = "/content/AccessibleLLM/object detection/dataset3/dataset3/model2d3withimg.txt"

# Process images and texts
process_images_and_texts(image_folder, text_file)

CLIP Score for 1.jpg: 0.291645884513855
CLIP Score for 10.jpg: 0.3104192614555359
CLIP Score for 11.jpg: 0.29700493812561035
CLIP Score for 12.jpg: 0.24349626898765564
CLIP Score for 13.jpg: 0.2932666540145874
CLIP Score for 14.jpg: 0.2544335722923279
CLIP Score for 15.jpg: 0.24630892276763916
CLIP Score for 16.jpg: 0.26716098189353943
CLIP Score for 17.jpg: 0.2593941390514374
CLIP Score for 18.jpg: 0.2776206135749817
CLIP Score for 19.jpg: 0.19640424847602844
CLIP Score for 2.jpg: 0.28858688473701477
CLIP Score for 20.jpg: 0.31447237730026245
CLIP Score for 21.jpg: 0.3279736042022705
CLIP Score for 22.jpg: 0.20974622666835785
CLIP Score for 23.jpg: 0.3018435835838318
CLIP Score for 24.jpg: 0.23844589293003082
CLIP Score for 25.jpg: 0.2424202561378479
CLIP Score for 26.jpg: 0.29085275530815125
CLIP Score for 27.jpg: 0.26420971751213074
CLIP Score for 28.jpg: 0.21436549723148346
CLIP Score for 29.jpg: 0.3132898509502411
CLIP Score for 3.jpg: 0.3059704005718231
CLIP Score for 30.jpg: 0.2

In [21]:
#scaled up scores for BLIP-2
import torch
import clip
from PIL import Image
import os

def get_clip_score(image_path, text):
    # Load the pre-trained CLIP model
    model, preprocess = clip.load('ViT-B/32')

    # Preprocess the image and tokenize the text
    image = Image.open(image_path)
    image_input = preprocess(image).unsqueeze(0)
    text_input = clip.tokenize([text])

    # Move the inputs to GPU if available
    device = "cuda" if torch.cuda.is_available() else "cpu"
    image_input = image_input.to(device)
    text_input = text_input.to(device)
    model = model.to(device)

    # Generate embeddings for the image and text
    with torch.no_grad():
        image_features = model.encode_image(image_input)
        text_features = model.encode_text(text_input)

    # Normalize the features
    image_features = image_features / image_features.norm(dim=-1, keepdim=True)
    text_features = text_features / text_features.norm(dim=-1, keepdim=True)

    # Calculate the cosine similarity to get the CLIP score
    clip_score = torch.matmul(image_features, text_features.T).item()

    # Scale the score to a range between 0 and 100
    scaled_score = (clip_score + 1) * 50

    return scaled_score

def process_images_and_texts(image_folder, text_file):
    # Get a list of image files
    image_files = [file for file in os.listdir(image_folder) if file.endswith((".jpg", ".jpeg", ".png"))]
    image_files.sort()  # Sort the list for consistent processing order

    # Read lines from the text file
    with open(text_file, 'r') as f:
        text_lines = f.readlines()

    # Process each image and text pair
    for image_file in image_files:
        # Find corresponding text line in the text file
        image_name = os.path.splitext(image_file)[0]
        for line in text_lines:
            if line.startswith(image_name):
                _, text = line.split(",", 1)  # Split by first comma to separate image name and text
                text = text.strip()  # Remove leading/trailing whitespaces
                image_path = os.path.join(image_folder, image_file)
                score = get_clip_score(image_path, text)
                print(f"CLIP Score for {image_file}: {score:.2f}")
                break
        else:
            print(f"No text found for {image_file}")

# Paths to image folder and text file
image_folder = "/content/AccessibleLLM/object detection/dataset3/dataset3"
text_file = "/content/AccessibleLLM/object detection/dataset3/dataset3/model1d3withimg.txt"

# Process images and texts
process_images_and_texts(image_folder, text_file)


CLIP Score for 1.jpg: 64.91
CLIP Score for 10.jpg: 66.08
CLIP Score for 11.jpg: 65.31
CLIP Score for 12.jpg: 61.77
CLIP Score for 13.jpg: 66.93
CLIP Score for 14.jpg: 66.06
CLIP Score for 15.jpg: 62.46
CLIP Score for 16.jpg: 65.64
CLIP Score for 17.jpg: 64.02
CLIP Score for 18.jpg: 65.02
CLIP Score for 19.jpg: 61.96
CLIP Score for 2.jpg: 64.87
CLIP Score for 20.jpg: 67.93
CLIP Score for 21.jpg: 68.59
CLIP Score for 22.jpg: 65.56
CLIP Score for 23.jpg: 65.23
CLIP Score for 24.jpg: 62.85
CLIP Score for 25.jpg: 62.70
CLIP Score for 26.jpg: 65.01
CLIP Score for 27.jpg: 67.61
CLIP Score for 28.jpg: 67.03
CLIP Score for 29.jpg: 66.49
CLIP Score for 3.jpg: 65.84
CLIP Score for 30.jpg: 64.62
CLIP Score for 31.jpg: 65.14
CLIP Score for 32.jpg: 65.70
CLIP Score for 33.jpg: 64.61
CLIP Score for 34.jpg: 64.69
CLIP Score for 35.jpg: 62.96
CLIP Score for 36.jpg: 66.54
CLIP Score for 37.jpg: 62.89
CLIP Score for 38.jpg: 68.52
CLIP Score for 39.jpg: 65.24
CLIP Score for 4.jpg: 67.44
CLIP Score for 40.

In [22]:
#xcaled up scores for VitGpt
import torch
import clip
from PIL import Image
import os

def get_clip_score(image_path, text):
    # Load the pre-trained CLIP model
    model, preprocess = clip.load('ViT-B/32')

    # Preprocess the image and tokenize the text
    image = Image.open(image_path)
    image_input = preprocess(image).unsqueeze(0)
    text_input = clip.tokenize([text])

    # Move the inputs to GPU if available
    device = "cuda" if torch.cuda.is_available() else "cpu"
    image_input = image_input.to(device)
    text_input = text_input.to(device)
    model = model.to(device)

    # Generate embeddings for the image and text
    with torch.no_grad():
        image_features = model.encode_image(image_input)
        text_features = model.encode_text(text_input)

    # Normalize the features
    image_features = image_features / image_features.norm(dim=-1, keepdim=True)
    text_features = text_features / text_features.norm(dim=-1, keepdim=True)

    # Calculate the cosine similarity to get the CLIP score
    clip_score = torch.matmul(image_features, text_features.T).item()

    # Scale the score to a range between 0 and 100
    scaled_score = (clip_score + 1) * 50

    return scaled_score

def process_images_and_texts(image_folder, text_file):
    # Get a list of image files
    image_files = [file for file in os.listdir(image_folder) if file.endswith((".jpg", ".jpeg", ".png"))]
    image_files.sort()  # Sort the list for consistent processing order

    # Read lines from the text file
    with open(text_file, 'r') as f:
        text_lines = f.readlines()

    # Process each image and text pair
    for image_file in image_files:
        # Find corresponding text line in the text file
        image_name = os.path.splitext(image_file)[0]
        for line in text_lines:
            if line.startswith(image_name):
                _, text = line.split(",", 1)  # Split by first comma to separate image name and text
                text = text.strip()  # Remove leading/trailing whitespaces
                image_path = os.path.join(image_folder, image_file)
                score = get_clip_score(image_path, text)
                print(f"CLIP Score for {image_file}: {score:.2f}")
                break
        else:
            print(f"No text found for {image_file}")

# Paths to image folder and text file
image_folder = "/content/AccessibleLLM/object detection/dataset3/dataset3"
text_file = "/content/AccessibleLLM/object detection/dataset3/dataset3/model2d3withimg.txt"

# Process images and texts
process_images_and_texts(image_folder, text_file)

CLIP Score for 1.jpg: 64.58
CLIP Score for 10.jpg: 65.52
CLIP Score for 11.jpg: 64.85
CLIP Score for 12.jpg: 62.17
CLIP Score for 13.jpg: 64.66
CLIP Score for 14.jpg: 62.72
CLIP Score for 15.jpg: 62.32
CLIP Score for 16.jpg: 63.36
CLIP Score for 17.jpg: 62.97
CLIP Score for 18.jpg: 63.88
CLIP Score for 19.jpg: 59.82
CLIP Score for 2.jpg: 64.43
CLIP Score for 20.jpg: 65.72
CLIP Score for 21.jpg: 66.40
CLIP Score for 22.jpg: 60.49
CLIP Score for 23.jpg: 65.09
CLIP Score for 24.jpg: 61.92
CLIP Score for 25.jpg: 62.12
CLIP Score for 26.jpg: 64.54
CLIP Score for 27.jpg: 63.21
CLIP Score for 28.jpg: 60.72
CLIP Score for 29.jpg: 65.66
CLIP Score for 3.jpg: 65.30
CLIP Score for 30.jpg: 62.51
CLIP Score for 31.jpg: 60.74
CLIP Score for 32.jpg: 66.16
CLIP Score for 33.jpg: 62.90
CLIP Score for 34.jpg: 64.77
CLIP Score for 35.jpg: 65.96
CLIP Score for 36.jpg: 65.58
CLIP Score for 37.jpg: 61.71
CLIP Score for 38.jpg: 63.07
CLIP Score for 39.jpg: 62.97
CLIP Score for 4.jpg: 63.93
CLIP Score for 40.